In [1]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

In [2]:
# Load the smallest OPT model to disk

model = "facebook/opt-125m"

# !mkdir offload_folder
model_kwargs = {"device_map": "cuda:0"}
m = AutoModelForCausalLM.from_pretrained(model, **model_kwargs, )
tokenizer = AutoTokenizer.from_pretrained(model, use_fast=False)
generator = pipeline(task="text-generation", model=m, tokenizer=tokenizer)

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/251M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [3]:
print(m)
inputs = tokenizer("Hugging Face is pushing the convention that a unicorn with two horns becomes a llama.", return_tensors="pt")

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 768, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 768)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-11): 12 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), ep

In [5]:
out = m(inputs["input_ids"].to(0))
inputs["input_ids"].to(0).shape

torch.Size([1, 19])

In [6]:
global_input = []
def fwd_hook(module,inputs,outputs):
  global_input.append(inputs)
  print("Executed hook successfully for decoder 1!")
  return outputs

In [7]:
module_to_hook = m.model.decoder.layers[0]
hook = module_to_hook.register_forward_hook(fwd_hook)

In [8]:
input_ids = inputs['input_ids'].to(0)

In [9]:
outputs = m(input_ids)

Executed hook successfully for decoder 1!


In [13]:
#This is the input to OPTDecoderBlock
print(global_input[0][0].shape)
hidden_states = global_input[0][0]

torch.Size([1, 19, 768])


In [12]:
import torch

In [14]:
x = hidden_states
mask = torch.ones((x.size(0), x.size(1)), dtype=torch.bool, device=x.device)
print(f"x shape {x.shape}")
print(f"Mask shape {mask.shape}")

x shape torch.Size([1, 19, 768])
Mask shape torch.Size([1, 19])


In [15]:
hidden_states.view(-1, hidden_states.size(-1)).shape

torch.Size([19, 768])

In [16]:
hidden_states.size(-1)

768

In [17]:
mask.bool().view(-1)

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True], device='cuda:0')

In [18]:
out.logits.shape

torch.Size([1, 19, 50272])

In [26]:
# This is from the actual example
inputs = tokenizer("Hugging Face is pushing the convention that a unicorn with two horns becomes a llama.", return_tensors="pt")

output = m.generate(inputs["input_ids"].to(0), min_length=30, max_length=3000, do_sample=True)

Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for decoder 1!
Executed hook successfully for dec

In [27]:
output.shape

torch.Size([1, 149])

In [28]:
print(tokenizer.decode(output[0].tolist()))

</s>Hugging Face is pushing the convention that a unicorn with two horns becomes a llama. For example, the baby unicorn can be "circled" by three horns, with the third one having horns.  I have 3 horns and I am an expert in unicorns. I will give this advice based on what I know about unicorns, especially because this is exactly the advice that's not going to work with the current convention.
I'm pretty new to all this   I have 3 horns and I can do this  I have 3 horns.
I bet you are a unicorn. You will win this thing.
I will give this advice based on what I know. You will win this one, too.</s>


In [23]:
inputs.input_ids.shape

torch.Size([1, 19])